In [1]:
from ads_query_eval.config import get_terminus_client
from ads_query_eval.app.bootstrap import QUERIES

client = get_terminus_client()

Checking for 200 response from http://terminus:6363/api/info...


In [3]:
import secrets

client.insert_document({
    "@type": "InviteLink",
    "token": secrets.token_urlsafe(),
})

['terminusdb:///data/InviteLink/LoFgwWinpD25eH9YVddCFgbiFnZF_y1kwdR3ABd01p0']

In [25]:
from ads_query_eval.lib.io import find_one

ecr = find_one(client, {"@type": "EmailedCredentialsLink"})
user = find_one(client, {"@type": "User", "@id": ecr['user']})
cr = find_one(client, {"@type": "CredentialsRequest", "@id": ecr['credentials_request']})
il = find_one(client, {"@type": "InviteLink", "@id": cr['invite_link']})
ecr, user, cr, il

({'@id': 'EmailedCredentialsLink/f1d07c99a2df661aaccbbf254a4bad3b720c27d97a4c8226278724fe7c2e0b82',
  '@type': 'EmailedCredentialsLink',
  'credentials_request': 'CredentialsRequest/38e6f46b7c3108f5843c2087cac9492bd7fdd9f3812f2f7394e68585724d8b68',
  'one_time_link': 'http://localhost:5000/snappassfbc53fb068e34fb09ca9a8ee76d9b264~Iz0PS3gbowidDUdDP1NLAqV8L3VE9FjwOPs79MSTNlk%3D',
  'user': 'User/d1c9781c55fc7f40246f2dfcee3b35d36a26fc3eda55d4efdebadf787bca21ca'},
 {'@id': 'User/d1c9781c55fc7f40246f2dfcee3b35d36a26fc3eda55d4efdebadf787bca21ca',
  '@type': 'User',
  'email_address': 'donny@polyneme.xyz',
  'hashed_password': '$2b$12$HhMcxyymOEBi431N7u1E7.GhVTu79YoIxls.OmUovbByye141bDFG',
  'username': 'd1c9781c55fc7f40246f2dfcee3b35d36a26fc3eda55d4efdebadf787bca21ca'},
 {'@id': 'CredentialsRequest/38e6f46b7c3108f5843c2087cac9492bd7fdd9f3812f2f7394e68585724d8b68',
  '@type': 'CredentialsRequest',
  'email_address': 'donny@polyneme.xyz',
  'invite_link': 'InviteLink/LoFgwWinpD25eH9YVddCFgbiFn

In [27]:
client.get_documents_by_type("InviteLink", as_list=True)

[{'@id': 'InviteLink/LoFgwWinpD25eH9YVddCFgbiFnZF_y1kwdR3ABd01p0',
  '@type': 'InviteLink',
  'token': 'LoFgwWinpD25eH9YVddCFgbiFnZF_y1kwdR3ABd01p0'}]

In [ ]:
client.replace_document([
    {
        "@type": "Query",
        "query_literal": 'full:"coronal mass ejection"',
        "@capture": "id_query_cme"
    },
    {
        "@type": "Retrieval",
        "query": {"@ref": "id_query_cme"},
        "status": "completed",
        "done": True,
        "@capture": "id_retrieval_cme"
    },
    {
        "@type": "RetrievedItemsList",
        "retrieval": {"@ref": "id_retrieval_cme"},
        "@capture": "id_retrieved_items_list_cme"
    },
    {
        "retrieved_items_list": {"@ref": "id_retrieved_items_list_cme"},
        "retrievable_item": {"@ref": "id_item_2022npjAM..14...43F"}
    },
    {
        "ads_bibcode": "2022npjAM..14...43F",
        "@capture": "id_item_2022npjAM..14...43F"
    },
    {
        "fqn": "ads_query_eval.frame.evaluators.hal",
        "version": "0.1",
        "config": {"mode": "awesome"},
        "@capture": "id_evaluator_hal"
    },
    {
        "evaluator": {"@ref": "id_evaluator_hal"},
        "p_at_25": 0.55,
        "r_at_1000": 0.10,
        "done": True,
        "status": "completed",
        "@capture": "id_evaluation_hal"
    },
    {
        "category": "SensedEvaluationOfRetrievedItemsByProcedure",
        "operations": [{"@ref": "id_evaluation_hal"}]
    },
    {
        "category": "ScheduledRetrieval",
        "operations": [{"@ref": "id_retrieval_cme"}]
    }
], commit_msg="Adding test docs", create=True)


list(client.get_all_documents())

In [ ]:
import requests

def tget(url):
    client._check_connection()
    return requests.get(
        url,
        headers=client._default_headers,
        auth=client._auth(),
    )

def tget_ttl(dbid, which=None):
    assert which in {"schema","instance"}
    rv = tget(f"http://terminus:6363/api/triples/admin/{dbid}/local/branch/main/{which}?format=turtle")
    for line in rv.json().split('\n'):
        print(line)
    return rv

tget_ttl(client.db, which="schema")
tget_ttl(client.db, which="instance")